In [29]:
import pandas as pd
import numpy as np
import os
directory = 'modelos/st'
if not os.path.exists(directory):
    os.makedirs(directory)
# Guardo modelo como un pickle file 
import joblib
from sklearn.metrics import mean_squared_error

In [2]:
auctions = pd.read_csv('../datos/auctions.csv', low_memory=False)

# Preparo el df y divido ventanas

In [3]:
auctions['date'] = pd.to_datetime(auctions['date'])

In [4]:
w1 = auctions[auctions['date'].dt.day < 21]
w2 = auctions[(auctions['date'].dt.day > 18) & (auctions['date'].dt.day < 22)]
w3 = auctions[(auctions['date'].dt.day > 19) & (auctions['date'].dt.day < 23)]
w4 = auctions[(auctions['date'].dt.day > 20) & (auctions['date'].dt.day < 24)]
w5 = auctions[(auctions['date'].dt.day > 21) & (auctions['date'].dt.day < 25)]
w6 = auctions[(auctions['date'].dt.day > 22) & (auctions['date'].dt.day < 26)]
w7 = auctions[(auctions['date'].dt.day > 23) & (auctions['date'].dt.day < 27)]

In [5]:
windowStart = pd.to_datetime("2019-04-18 00:00:00.000000")
w1['timeFromWindowStart'] = (w1['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-19 00:00:00.000000")
w2['timeFromWindowStart'] = (w2['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-20 00:00:00.000000")
w3['timeFromWindowStart'] = (w3['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-21 00:00:00.000000")
w4['timeFromWindowStart'] = (w4['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-22 00:00:00.000000")
w5['timeFromWindowStart'] = (w5['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-23 00:00:00.000000")
w6['timeFromWindowStart'] = (w6['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-23 00:00:00.000000")
w7['timeFromWindowStart'] = (w7['date'] - windowStart)/np.timedelta64(1,'s')

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [6]:
w1 = w1.sort_values(by=['device_id', 'date'])
w1 = w1.drop_duplicates('device_id', keep = 'first')
w2 = w2.sort_values(by=['device_id', 'date'])
w2 = w2.drop_duplicates('device_id', keep = 'first')
w3 = w3.sort_values(by=['device_id', 'date'])
w3 = w3.drop_duplicates('device_id', keep = 'first')
w4 = w4.sort_values(by=['device_id', 'date'])
w4 = w4.drop_duplicates('device_id', keep = 'first')
w5 = w5.sort_values(by=['device_id', 'date'])
w5 = w5.drop_duplicates('device_id', keep = 'first')
w6 = w6.sort_values(by=['device_id', 'date'])
w6 = w6.drop_duplicates('device_id', keep = 'first')
w7 = w7.sort_values(by=['device_id', 'date'])
w7 = w7.drop_duplicates('device_id', keep = 'first')

In [7]:
w1.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w2.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w3.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w4.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w5.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w6.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w7.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']

In [8]:
w1 = w1.drop(columns = ['date'])
w2 = w2.drop(columns = ['date'])
w3 = w3.drop(columns = ['date'])
w4 = w4.drop(columns = ['date'])
w5 = w5.drop(columns = ['date'])
w6 = w6.drop(columns = ['date'])
w7 = w7.drop(columns = ['date'])

# Explicación

Para calcular finalmente con XGBoost, nosotros agregamos predicciones con otros métodos como features así que
vamos a usar el entrenamiento que hicimos durante el análisis exploratorio para predecir estos features y poder
agregar las columnas que se necesitan para la predicción final.

## Aplico KNN y RandomForest 
Aplico KNN y RandomForest en la primer ventana y hago las predicciones de la cuarta ventana y despues le voy a agregar esas predicciones como features.

In [9]:
X, y = w1.iloc[:,:-1],w1.iloc[:,-1]

In [14]:
neigh = joblib.load('modelos/st/neigh.pkl')

In [15]:
preds_KNN_w4 = neigh.predict(w4.iloc[:,:-1])

In [16]:
rf = joblib.load('modelos/st/rf.pkl')

In [17]:
preds_RF_w4 = rf.predict(w4.iloc[:,:-1])

## Entreno con w4
Ya me guarde las predicciones de w4 de RF y de KNN y las voy a agregar como features luego.
Ahora voy a entrenar con w4 y validar con w7 y luego les voy a agregar tanto a w4 como a w7 los features de las predicciones de KNN y de RF

In [18]:
preds_KNN_w7 = neigh.predict(w7.iloc[:,:-1])

In [19]:
preds_RF_w7 = rf.predict(w7.iloc[:,:-1])

In [20]:
w4["pred_KNN"] = preds_KNN_w4

In [21]:
w4["pred_RF"] = preds_RF_w4
w7["pred_KNN"] = preds_KNN_w7
w7["pred_RF"] = preds_RF_w7

In [22]:
w4.columns = ['device_id', 'ref_type_id', 'source_id', 'pred_KNN', 'pred_RF', 'st']
w7.columns = ['device_id', 'ref_type_id', 'source_id', 'pred_KNN', 'pred_RF', 'st']

In [23]:
w4.head()

,device_id,ref_type_id,source_id,pred_KNN,pred_RF,st
36601349,40621409780134,1,0,226857.222979,89942.06188,83109.410677
33917684,168103949904656,1,3,173044.526198,89942.06188,82621.700990
9402375,186034136943920,1,1,88.851558,89942.06188,79298.492452
13736755,295841792051458,1,1,432.827168,89942.06188,79298.492452
43932908,345999128501141,1,0,164703.549846,89942.06188,83109.410677


In [24]:
w7.head()

,device_id,ref_type_id,source_id,pred_KNN,pred_RF,st
39675750,69039685746313,1,7,212658.597103,89942.06188,84232.355371
1528620,345999128501141,1,2,336762.048531,89942.06188,88099.527648
20706804,360710529886978,1,1,133087.256609,89942.06188,79298.492452
35163923,365882020742330,1,8,183027.763854,89942.06188,168363.440468
20794330,416301579449694,1,1,98969.446617,89942.06188,79298.492452


## Ahora voy a entrenar w4 y validar con w7 con XBBoost con las nuevas features

In [25]:
X, y = w4.iloc[:,:-1],w4.iloc[:,-1]

In [26]:
xg_reg = joblib.load('modelos/st/xg_reg.pkl')

[12:19:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [27]:
preds = xg_reg.predict(w7.iloc[:,:-1])

In [30]:
rmse = np.sqrt(mean_squared_error(w7.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 66070.668058


## Preparo el target_competencia

In [32]:
targets_st = pd.read_csv('../datos/target_competencia_ids.csv')
targets_st = targets_st[~targets_st['ref_hash'].str.contains('_sc')]
targets_st['ref_hash'] = targets_st['ref_hash'].map(lambda x: str(x)[:-3])

In [33]:
targets_st['ref_hash'] = targets_st['ref_hash'].astype(np.int64)
targets_st.columns = ['device_id', 'date_dif']

In [34]:
auctions_without_duplicates = auctions.drop_duplicates(subset = 'device_id')

In [35]:
targets_st = targets_st.merge(auctions_without_duplicates[['device_id', 'ref_type_id', 'source_id']], on = 'device_id', how = 'left')

In [36]:
targets_st = targets_st.drop(columns = ['date_dif'])

Ahora que tengo este target armado con los features de auctions y filtrado por los que necesito, lo guardo en un csv para poder reutilizar esto en una próxima ocasión

## Predigo con KNN y con RF target compentencia para agregárcelos como features

In [39]:
w7 = w7.drop(columns = ['pred_KNN'])
w7 = w7.drop(columns = ['pred_RF'])

In [40]:
w7.head()

,device_id,ref_type_id,source_id,st
39675750,69039685746313,1,7,84232.355371
1528620,345999128501141,1,2,88099.527648
20706804,360710529886978,1,1,79298.492452
35163923,365882020742330,1,8,168363.440468
20794330,416301579449694,1,1,79298.492452


In [41]:
X, y = w7.iloc[:,:-1],w7.iloc[:,-1]

In [42]:
preds_KNN_targets = neigh.predict(targets_st)

In [43]:
preds_RF_targets = rf.predict(targets_st)

In [44]:
w7["pred_KNN"] = preds_KNN_w7
w7["pred_RF"] = preds_RF_w7
targets_st["pred_KNN"] = preds_KNN_targets
targets_st["pred_RF"] = preds_RF_targets

In [45]:
w7.columns = ['device_id', 'ref_type_id', 'source_id', 'pred_KNN', 'pred_RF', 'st']

In [46]:
w7.head()

,device_id,ref_type_id,source_id,pred_KNN,pred_RF,st
39675750,69039685746313,1,7,84232.355371,89942.06188,84232.355371
1528620,345999128501141,1,2,88099.527648,89942.06188,88099.527648
20706804,360710529886978,1,1,79298.492452,89942.06188,79298.492452
35163923,365882020742330,1,8,168363.440468,89942.06188,168363.440468
20794330,416301579449694,1,1,79298.492452,89942.06188,79298.492452


In [52]:
targets_st.head()

,device_id,ref_type_id,source_id,pred_KNN,pred_RF
0,1000169251625791246,1,1,81316.749726,79298.492452
1,1000395625957344683,1,8,81094.796346,92193.074740
2,1003027494996471685,1,0,81768.118050,83109.410677
3,1006670001679961544,7,0,80504.595306,83109.410677
4,1007573308966476713,1,1,81028.109010,79298.492452


In [54]:
targets_st.to_csv('modelos/st/targets_st.csv', index=False)

## Ahora aplico XGBoost 
Aplico XGBoost previamente entrendado con w7 y prediciendo targets

In [49]:
preds = xg_reg.predict(targets_st)

In [50]:
for pred in preds: print(pred)

79299.82
79931.26
83127.5
80217.84
79299.82
83127.5
83134.82
82209.48
82209.48
79299.82
79299.82
79708.9
80334.48
83127.5
82209.48
79218.19
79709.84
80217.84
82216.8
83127.5
82619.5
80334.48
82209.48
82209.48
87877.23
79299.82
79298.875
80217.84
79299.82
79299.82
82209.48
82209.48
79298.875
84935.99
83127.5
82209.48
83127.5
82209.48
83134.82
82619.5
83127.5
82209.48
82209.48
79299.82
79709.84
81579.45
82208.53
79299.82
83127.5
79299.82
84935.05
80217.84
82619.5
80217.84
82744.44
82209.48
82209.48
82209.48
82209.48
82209.48
79299.82
79299.82
77424.81
79299.82
80217.84
80334.48
82208.53
79299.82
80217.84
79299.82
79299.82
79709.84
79931.26
79299.82
79299.82
82209.48
79318.55
82209.48
82861.91
84935.99
82619.5
83127.5
83127.5
82209.48
79299.82
79299.82
82209.48
82619.5
83127.5
80334.48
77424.81
77424.81
82209.48
79834.77
82848.234
82209.48
83127.5
82209.48
77424.81
79299.82
82209.48
79299.82
84489.12
83127.5
84935.99
83127.5
80217.84
79299.82
82209.48
83127.5
84935.99
82209.48
83127.5
792

80334.48
79299.82
80217.84
82744.44
80217.84
82619.5
82209.48
82209.48
82619.5
79299.82
82209.48
82744.44
80217.84
83127.5
79299.82
82619.5
81446.01
82208.53
82209.48
77424.81
79299.82
78536.34
81579.45
77424.81
83134.82
83174.81
80217.84
79299.82
79299.82
82814.59
82209.48
83127.5
80378.36
81579.45
82209.48
82209.48
79299.82
82814.59
84489.12
79299.82
79709.84
79299.82
82209.48
82209.48
80334.48
82619.5
79299.82
80217.84
80341.8
83174.81
82619.5
80334.48
80334.48
79299.82
82744.44
79299.82
82126.9
82209.48
84954.72
82209.48
82209.48
82619.5
79298.875
79620.75
79708.9
79299.82
80217.84
80159.62
82209.48
84489.12
80334.48
82209.48
82814.59
80334.48
84489.12
82209.48
82209.48
82209.48
83127.5
84489.12
79299.82
82216.8
82216.8
82209.48
83127.5
79299.82
82209.48
82530.41
82209.48
82209.48
80217.84
82209.48
82209.48
82216.8
84935.99
82026.336
80217.84
82209.48
84935.99
82209.48
82209.48
82209.48
83127.5
84489.12
83127.5
77424.81
79299.82
80217.84
82209.48
82209.48
83174.81
82209.48
79620.75

82666.81
80341.8
82530.41
82209.48
82530.41
79299.82
79299.82
79299.82
82209.48
82209.48
82209.48
83127.5
82209.48
85066.195
79299.82
79709.84
79709.84
82209.48
82619.5
83127.5
80334.48
82619.5
82209.48
82209.48
82209.48
82619.5
82619.5
80334.48
80217.84
80217.84
82209.48
82209.48
82209.48
82209.48
82209.48
82209.48
83127.5
79299.82
81617.74
79299.82
82209.48
82209.48
82208.53
82209.48
82208.53
80217.84
82209.48
79709.84
82209.48
78536.34
79709.84
80217.84
79299.82
83126.555
82530.41
82209.48
82209.48
82209.48
77424.81
82814.59
80334.48
82209.48
82209.48
79299.82
79298.875
80334.48
83127.5
79299.82
82156.54
82209.48
82209.48
82209.48
83127.5
83127.5
83127.5
79299.82
80217.84
82209.48
82209.48
82209.48
83127.5
82216.8
82209.48
79299.82
82156.54
80284.45
82530.41
83127.5
85066.195
85066.195
77424.81
79299.82
78536.34
79299.82
79299.82
83127.5
79299.82
79709.84
79299.82
79709.84
82209.48
79299.82
79299.82
82530.41
82209.48
82216.8
82126.9
82209.48
79299.82
84527.41
82209.48
82626.82
82216

79299.82
80217.84
77056.93
82683.29
79904.94
79299.82
82209.48
82209.48
79299.82
79709.84
79709.84
80217.84
77056.93
79709.84
79773.63
83127.5
82209.48
82209.48
79709.84
77056.93
82209.48
82814.59
84935.95
82209.44
82619.5
82209.48
83127.5
80217.84
79709.84
79299.82
79773.63
82619.5
82209.48
82834.086
82814.59
82209.48
82683.29
83127.5
79299.82
82209.44
79299.82
82209.44
81617.74
79299.82
83127.5
82209.48
79966.586
79299.82
79299.82
79299.82
79299.82
79299.82
82814.59
82209.48
79299.82
83127.5
82209.48
79299.82
79299.82
80217.84
79966.586
82209.48
82209.48
83127.5
82209.48
77056.93
80217.84
82209.44
79299.82
77374.79
79299.82
79299.82
79299.82
79299.82
79299.82
79966.586
79966.586
82209.48
82209.48
79299.82
79299.82
79299.82
83127.5
82626.82
79299.82
79299.82
79299.82
79299.82
83127.5
84660.88
82209.44
79299.82
79299.82
79299.82
84527.41
79299.82
80217.84
82209.48
81617.74
79299.82
79299.82
79299.82
80217.84
79388.05
79904.94
79904.94
79299.82
79299.82
80217.84
79299.82
79388.05
83127.

In [ ]:
pd.DataFrame(preds).to_csv("../output/predicciones_st.csv", header=None, index=False)